In [1]:
import importlib
import frengression
# importlib.reload(frengression)
from data_causl.utils import *
from data_causl.data import *
from frengression import *

device = torch.device('cpu')
from CausalEGM import *

import numpy as np
import jax.numpy as jnp
import pickle
import os
from tqdm import tqdm
import src.exp_utils as exp_utils

from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import log_loss
from scipy.spatial.distance import pdist, squareform
import warnings

warnings.filterwarnings("ignore")


In [18]:
s, x,z,y= generate_data_survivl(n=200,T=2)

In [19]:
s = torch.tensor(s, dtype=torch.float32)
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
z = torch.tensor(z, dtype=torch.float32)

In [27]:
model = FrengressionSeq(x_dim=1, y_dim=1, z_dim=1, T=2, s_dim = 1, noise_dim=10, num_layer=3, hidden_dim=100, device=device, x_binary = False)

In [28]:
model.train_xz(x=x, z=z, y=y, s=s, num_iters=6000, lr=1e-3, print_every_iter=1000)

Epoch 1: loss 1.9284, loss1 2.0196, loss2 0.1823
Epoch 1000: loss 1.1151, loss1 2.1733, loss2 2.1163
Epoch 2000: loss 1.0514, loss1 2.1108, loss2 2.1189
Epoch 3000: loss 0.9662, loss1 2.0495, loss2 2.1665
Epoch 4000: loss 0.9108, loss1 1.9159, loss2 2.0101
Epoch 5000: loss 0.8897, loss1 1.7543, loss2 1.7292
Epoch 6000: loss 0.9172, loss1 1.7720, loss2 1.7097


In [22]:
model.train_y(x=x,z=z,y=y,num_iters=12000, lr=1e-3, print_every_iter=1000)

Epoch 1: loss 2.6314,	loss_y 1.5334, 1.5794, 0.0922,	loss_eta 1.0980, 1.1614, 0.1267
Epoch 1000: loss 2.1286,	loss_y 1.1981, 2.9719, 3.5477,	loss_eta 0.9305, 2.3411, 2.8211
Epoch 2000: loss 2.2698,	loss_y 1.1908, 1.9776, 1.5735,	loss_eta 1.0789, 2.6841, 3.2102
Epoch 3000: loss 1.9792,	loss_y 1.0433, 2.1132, 2.1397,	loss_eta 0.9358, 3.5247, 5.1778
Epoch 4000: loss 2.2281,	loss_y 1.2250, 3.3278, 4.2056,	loss_eta 1.0031, 5.4816, 8.9569
Epoch 5000: loss 2.8970,	loss_y 1.3488, 2.2971, 1.8965,	loss_eta 1.5482, 5.4872, 7.8780
Epoch 6000: loss 2.3019,	loss_y 1.0413, 2.2039, 2.3252,	loss_eta 1.2606, 1.9614, 1.4016
Epoch 7000: loss 2.4458,	loss_y 1.1734, 2.6794, 3.0119,	loss_eta 1.2724, 2.9953, 3.4459
Epoch 8000: loss 2.4566,	loss_y 1.1607, 2.6394, 2.9575,	loss_eta 1.2959, 1.8396, 1.0874
Epoch 9000: loss 2.4973,	loss_y 1.1985, 2.7726, 3.1482,	loss_eta 1.2989, 2.3485, 2.0993
Epoch 10000: loss 2.5095,	loss_y 1.1633, 3.3920, 4.4575,	loss_eta 1.3462, 5.5093, 8.3261
Epoch 11000: loss 2.4147,	loss_y 1

In [29]:
model.sample_xz(s=s,x=x,z=z,y=y)

(tensor([[ 4.5711e-01,  5.5070e-01],
         [ 2.6892e+00,  9.8474e-01],
         [ 2.1487e+00,  4.8439e-01],
         [ 1.5602e+00,  1.2983e+00],
         [ 1.8080e+00,  1.0813e+00],
         [ 9.9904e-01, -3.8932e-03],
         [ 9.2676e-01,  2.3307e+00],
         [ 2.9032e+00,  3.1845e-01],
         [ 2.7167e+00,  2.9182e-01],
         [ 3.1017e-01,  1.3482e+00],
         [ 4.5099e-01,  8.6952e-01],
         [ 6.0183e-01,  5.0138e-01],
         [ 2.2388e+00,  2.3949e-01],
         [ 2.5959e+00,  8.1415e-01],
         [ 4.9220e-01,  1.2230e+00],
         [ 3.3708e-01,  4.6998e-01],
         [ 1.7184e-02,  6.4613e-01],
         [ 1.8429e+00,  1.6545e+00],
         [ 1.1709e+00,  6.4626e-01],
         [ 2.6360e+00,  6.7522e-01],
         [ 8.8619e-01,  3.6716e-01],
         [ 4.7743e-01,  1.1647e+00],
         [ 8.8935e-01,  1.3034e+00],
         [ 4.6997e-01,  5.8564e-01],
         [ 1.4103e+00,  6.0986e-01],
         [ 7.9688e-01,  1.2082e+00],
         [ 2.9273e-01,  8.0674e-01],
 

In [30]:
eta_try= model.sample_eta(x=x,z=z)
y_try= model.sample_y(x=x,z=z,eta=eta_try)

In [31]:
eta_try

tensor([[-0.0539,  0.1537],
        [ 0.0638,  0.1303],
        [-0.0458,  0.1489],
        [ 0.0464,  0.1468],
        [-0.0875,  0.2082],
        [-0.1160,  0.0878],
        [ 0.0488,  0.1127],
        [ 0.2080,  0.1604],
        [-0.0722,  0.2293],
        [ 0.1332,  0.1525],
        [ 0.0629,  0.0844],
        [ 0.0339,  0.1393],
        [ 0.0249,  0.2035],
        [-0.0488,  0.1414],
        [-0.0060,  0.2526],
        [ 0.0714,  0.0991],
        [-0.1031,  0.3373],
        [ 0.1511,  0.1407],
        [ 0.0523,  0.1577],
        [ 0.0122,  0.0598],
        [-0.0020,  0.1568],
        [ 0.0112,  0.1061],
        [ 0.0730,  0.2282],
        [ 0.0350,  0.0889],
        [ 0.0893,  0.0362],
        [ 0.0128,  0.1518],
        [ 0.0315,  0.0742],
        [-0.0558,  0.1272],
        [-0.0226,  0.1628],
        [-0.0720,  0.1526],
        [ 0.0313,  0.1756],
        [ 0.1551,  0.1842],
        [ 0.0404,  0.1846],
        [-0.0171,  0.1561],
        [-0.0324,  0.1200],
        [-0.0220,  0

In [32]:
y_try

tensor([[ 7.6963e-02, -9.0118e-02],
        [ 2.4040e-01,  6.3398e-02],
        [ 3.3286e-02,  5.8162e-02],
        [-5.4654e-02,  4.2347e-02],
        [ 1.8195e-01,  8.9477e-02],
        [-5.8436e-02, -3.9053e-02],
        [ 1.6702e-01,  8.1910e-02],
        [ 1.1742e-01,  1.4740e-01],
        [ 1.5494e-01,  1.2488e-01],
        [ 7.3922e-02,  1.2006e-01],
        [ 1.0871e-01,  5.8173e-02],
        [-2.0139e-02,  1.6039e-01],
        [ 5.1388e-02, -8.7264e-03],
        [ 1.9460e-01,  5.0270e-02],
        [-8.6262e-03,  6.0086e-02],
        [-2.0088e-02,  2.5292e-03],
        [ 1.1283e-01,  7.3010e-02],
        [ 2.0696e-01,  5.6392e-02],
        [ 2.6582e-02,  9.2377e-02],
        [ 1.1620e-01,  7.9592e-02],
        [ 4.2422e-02,  1.5477e-01],
        [ 4.5842e-02, -4.4836e-02],
        [ 6.1330e-02,  6.1491e-02],
        [-9.2808e-03,  4.2499e-02],
        [ 1.1748e-01,  1.4571e-01],
        [ 2.4758e-02,  1.1966e-01],
        [ 4.5742e-02,  1.6107e-01],
        [ 4.8631e-02,  6.901